In [1]:
import os 
import pandas as pd 
import numpy as np

from prediction_utils.pytorch_utils.metrics import StandardEvaluator

pd.options.display.max_rows = 600

In [2]:
counts_artifacts_fpath = '/labs/shahlab/projects/lguo/temp_ds_shift_robustness/clmbr/experiments/baseline/artifacts'
ete_artifacts_fpath = '/labs/shahlab/projects/lguo/temp_ds_shift_robustness/clmbr/experiments/clmbr/clmbr_artifacts'
clmbr_artifacts_fpath = '/labs/shahlab/projects/lguo/temp_ds_shift_robustness/clmbr/experiments/clmbr/adapter_artifacts'

In [3]:
n_boot = 1000

### Gather predictions

In [4]:
tasks = ['hospital_mortality','LOS_7','readmission_30','icu_admission']
nns = ['gru','transformer']
years = {
    '2013':'2013_2014_2015_2016',
    '2014':'2013_2014_2015_2016',
    '2015':'2013_2014_2015_2016',
    '2016':'2013_2014_2015_2016',
    '2017':'2017_2018_2019_2020_2021',
    '2018':'2017_2018_2019_2020_2021',
    '2019':'2017_2018_2019_2020_2021',
    '2020':'2017_2018_2019_2020_2021',
    '2021':'2017_2018_2019_2020_2021',
}

df_counts, df_clmbr, df_ete = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for task in tasks:

    # counts
    fpath = os.path.join(
        counts_artifacts_fpath,
        task,
        f'pred_probs/lr_2009_2010_2011_2012/'
    )

    filename = [x for x in os.listdir(fpath) if 'best_model' in x][0]
    df = pd.read_csv(os.path.join(fpath,filename))
    df['test_group'] = df['test_group'].astype(str)

    df = df.replace(years)

    # get prediction_id for counts-based models
    if task != 'readmission_30':
        index_ = 'admission'
    else:
        index_ = 'discharge'

    df_row_id_map = pd.read_parquet(
        os.path.join(
            f"/labs/shahlab/projects/lguo/temp_ds_shift_robustness/clmbr/features/admissions_{index_}_midnight/merged_features_binary/features_sparse",
            "features_row_id_map.parquet"
        )
    )

    df = df.merge(
        df_row_id_map,
        left_on='row_id',
        right_on='features_row_id'
    )


    df_counts = pd.concat((
        df_counts, 
        (
            df
            .query("test_group==['2009_2010_2011_2012','2013_2014_2015_2016','2017_2018_2019_2020_2021']")
            [['prediction_id','pred_probs','labels','task','test_group']]
            .assign(encoder='counts')
        )
    )).reset_index(drop=True)

    # clmbr & ete
    for model in nns:

        # clmbr
        fpath = os.path.join(
            clmbr_artifacts_fpath,
            task,
            f'pred_probs/{model}_lr_2009_2010_2011_2012/',
        )

        filename = [x for x in os.listdir(fpath) if 'best_model' in x][0]

        df = pd.read_csv(os.path.join(fpath,filename))
        df['test_group'] = df['test_group'].astype(str)

        df = df.replace(years)

        df_clmbr = pd.concat((
            df_clmbr, 
            (
                df
                .query("test_group==['2009_2010_2011_2012','2013_2014_2015_2016','2017_2018_2019_2020_2021']")
                [['prediction_id','pred_probs','labels','task','test_group']]
                .assign(encoder=model)
            )
        ))

        # ete
        fpath = os.path.join(
            ete_artifacts_fpath,
            'eval_end_to_end',
            task,
            f'{model}_2009_2010_2011_2012/',
            'pred_probs.csv'
        )

        df = pd.read_csv(fpath)
        df['test_group'] = df['test_group'].astype(str)

        df_ete = pd.concat((
            df_ete, 
            (
                df
                .query("test_group==['2009_2010_2011_2012','2013_2014_2015_2016','2017_2018_2019_2020_2021']")
                [['prediction_id','pred_probs','labels','test_group']]
                .assign(task=task,encoder=model+'_ete')
            )
        ))

df_counts = df_counts.sort_values(['task','prediction_id']).reset_index(drop=True)
df_gru = df_clmbr.query("encoder=='gru'").sort_values(['task','prediction_id']).reset_index(drop=True)
df_transf = df_clmbr.query("encoder=='transformer'").sort_values(['task','prediction_id']).reset_index(drop=True)
df_gru_ete = df_ete.query("encoder=='gru_ete'").sort_values(['task','prediction_id']).reset_index(drop=True)
df_transf_ete = df_ete.query("encoder=='transformer_ete'").sort_values(['task','prediction_id']).reset_index(drop=True)
        
assert(
    len(set(df_counts['prediction_id'].tolist())) ==
    len(set(df_gru['prediction_id'].tolist())) ==
    len(set(df_transf['prediction_id'].tolist())) ==
    len(set(df_gru_ete['prediction_id'].tolist())) ==
    len(set(df_transf_ete['prediction_id'].tolist()))
)

/labs/shahlab/envs/lguo/temp_ds_shift_robustness/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### ID Comparisons:
- counts vs gru-CLMBR
- counts vs transf-CLMBR
- gru-CLMBR vs gru-ete
- transf-CLMBR vs transf-ete
- gru-CLMBR vs transf-CLMBR

In [5]:
df_results = pd.DataFrame()

for year_group in ['2009_2010_2011_2012','2013_2014_2015_2016','2017_2018_2019_2020_2021']:
    for task in tasks:

        evaluator = StandardEvaluator(
            metrics=['auc','auprc_c','ace_abs_logistic_logit'],
            **{'pi0':df_counts.query("test_group=='2009_2010_2011_2012' and task==@task")['labels'].mean()}
        )

        for i_boot in range(n_boot):

            i_counts = (
                df_counts
                .query("task==@task and test_group==@year_group")
                .groupby("labels")
                .sample(frac=1,replace=True)
            )

            p_counts = evaluator.evaluate(i_counts)
            p_gru = evaluator.evaluate(df_gru.loc[i_counts.index,:])
            p_transf = evaluator.evaluate(df_transf.loc[i_counts.index,:])
            p_gru_ete = evaluator.evaluate(df_gru_ete.loc[i_counts.index,:])
            p_transf_ete = evaluator.evaluate(df_transf_ete.loc[i_counts.index,:])

            df_results = pd.concat((
                df_results,
                # counts vs gru
                pd.DataFrame({
                    'Year_Group':year_group,
                    'Comparison':'GRU-Counts',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru['performance']-p_counts['performance'],
                    'Task':task,
                    'Boot_Num':i_boot,
                }),
                #counts vs transformer
                pd.DataFrame({
                    'Year_Group':year_group,
                    'Comparison':'Transf-Counts',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_transf['performance']-p_counts['performance'],
                    'Task':task,
                    'Boot_Num':i_boot,
                }),
                #gru-ete vs gru-clmbr
                pd.DataFrame({
                    'Year_Group':year_group,
                    'Comparison':'GRU-GRU_ETE',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru['performance']-p_gru_ete['performance'],
                    'Task':task,
                    'Boot_Num':i_boot,
                }),
                #transformer-ete vs transformer-clmbr
                pd.DataFrame({
                    'Year_Group':year_group,
                    'Comparison':'Transf-Transf_ETE',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_transf['performance']-p_transf_ete['performance'],
                    'Task':task,
                    'Boot_Num':i_boot,
                }),
                #gru-clmbr vs transformer-clmbr
                pd.DataFrame({
                    'Year_Group':year_group,
                    'Comparison':'GRU-Transf',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru['performance']-p_transf['performance'],
                    'Task':task,
                    'Boot_Num':i_boot,
                }),
            ))

df_results.to_csv("tables/id.csv",index=False)

#### OOD Comparisons

In [6]:
df_results = pd.DataFrame()

for year_group in ['2013_2014_2015_2016','2017_2018_2019_2020_2021']:
    for task in tasks:

        evaluator = StandardEvaluator(
            metrics=['auc','auprc_c','ace_abs_logistic_logit'],
            **{'pi0':df_counts.query("test_group=='2009_2010_2011_2012' and task==@task")['labels'].mean()}
        )

        for i_boot in range(n_boot):
            
            # ID
            i_counts_id = (
                df_counts
                .query("task==@task and test_group=='2009_2010_2011_2012'")
                .groupby("labels")
                .sample(frac=1,replace=True)
            )

            p_counts_id = evaluator.evaluate(i_counts_id)
            p_gru_id = evaluator.evaluate(df_gru.loc[i_counts_id.index,:])
            p_transf_id = evaluator.evaluate(df_transf.loc[i_counts_id.index,:])
            p_gru_ete_id = evaluator.evaluate(df_gru_ete.loc[i_counts_id.index,:])
            p_transf_ete_id = evaluator.evaluate(df_transf_ete.loc[i_counts_id.index,:])
            
            # OOD
            i_counts_ood = (
                df_counts
                .query("task==@task and test_group==@year_group")
                .groupby("labels")
                .sample(frac=1,replace=True)
            )

            p_counts_ood = evaluator.evaluate(i_counts_ood)
            p_gru_ood = evaluator.evaluate(df_gru.loc[i_counts_ood.index,:])
            p_transf_ood = evaluator.evaluate(df_transf.loc[i_counts_ood.index,:])
            p_gru_ete_ood = evaluator.evaluate(df_gru_ete.loc[i_counts_ood.index,:])
            p_transf_ete_ood = evaluator.evaluate(df_transf_ete.loc[i_counts_ood.index,:])
            
            p_counts_ood=p_counts_ood.assign(delta=p_counts_ood['performance']-p_counts_id['performance'])
            p_gru_ood=p_gru_ood.assign(delta=p_gru_ood['performance']-p_gru_id['performance'])
            p_transf_ood=p_transf_ood.assign(delta=p_transf_ood['performance']-p_transf_id['performance'])
            p_gru_ete_ood=p_gru_ete_ood.assign(delta=p_gru_ete_ood['performance']-p_gru_ete_id['performance'])
            p_transf_ete_ood=p_transf_ete_ood.assign(delta=p_transf_ete_ood['performance']-p_transf_ete_id['performance'])

            df_results = pd.concat((
                df_results,
                # counts vs gru
                pd.DataFrame({
                    'Comparison':'GRU-Counts',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru_ood['delta']-p_counts_ood['delta'],
                    'Task':task,
                    'Boot_Num':i_boot,
                    'Year_Group':year_group,
                }),
                #counts vs transformer
                pd.DataFrame({
                    'Comparison':'Transf-Counts',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_transf_ood['delta']-p_counts_ood['delta'],
                    'Task':task,
                    'Boot_Num':i_boot,
                    'Year_Group':year_group,
                }),
                #gru-ete vs gru-clmbr
                pd.DataFrame({
                    'Comparison':'GRU-GRU_ETE',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru_ood['delta']-p_gru_ete_ood['delta'],
                    'Task':task,
                    'Boot_Num':i_boot,
                    'Year_Group':year_group,
                }),
                #transformer-ete vs transformer-clmbr
                pd.DataFrame({
                    'Comparison':'Transf-Transf_ETE',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_transf_ood['delta']-p_transf_ete_ood['delta'],
                    'Task':task,
                    'Boot_Num':i_boot,
                    'Year_Group':year_group,
                }),
                #gru-clmbr vs transformer-clmbr
                pd.DataFrame({
                    'Comparison':'GRU-Transf',
                    'Metric':['auc','auprc_c','ace_abs_logistic_logit'],
                    'Delta':p_gru_ood['delta']-p_transf_ood['delta'],
                    'Task':task,
                    'Boot_Num':i_boot,
                    'Year_Group':year_group,
                }),
            ))

df_results.to_csv("tables/ood.csv",index=False)

#### Tables

In [7]:
df_id = pd.read_csv("tables/id.csv")

df_id = (
    df_id
    .groupby(['Year_Group','Comparison','Task','Metric'])['Delta']
    .quantile([0.025,0.5,0.975])
    .unstack()
    .reset_index()
    .query("Metric==['auc','auprc_c','ace_abs_logistic_logit']")
    .rename(columns={0.5:'mid',0.025:'lower',0.975:'upper'})
)

df_id = (
    df_id
    .assign(
        Delta = df_id['mid'].round(3).astype(str) + 
        " (" + df_id['lower'].round(3).astype(str) + "," + 
        df_id['upper'].round(3).astype(str) + ")"
    )
    .pivot_table(
        columns=['Year_Group','Comparison'],
        index=['Task','Metric'],
        values='Delta',
        aggfunc='first'
    )
    .reindex(labels=tasks,level=0)
    .reindex(labels=['auc','auprc_c','ace_abs_logistic_logit'],level=1)
)

df_id = df_id[[
    (     '2009_2010_2011_2012',        'GRU-Counts'),
    (     '2009_2010_2011_2012',        'Transf-Counts'),
    (     '2009_2010_2011_2012',       'GRU-Transf'),
    (     '2009_2010_2011_2012',     'GRU-GRU_ETE'),
    (     '2009_2010_2011_2012', 'Transf-Transf_ETE'),
    (     '2013_2014_2015_2016',        'GRU-Counts'),
    (     '2013_2014_2015_2016',        'Transf-Counts'),
    (     '2013_2014_2015_2016',       'GRU-Transf'),
    (     '2013_2014_2015_2016',     'GRU-GRU_ETE'),
    (     '2013_2014_2015_2016', 'Transf-Transf_ETE'),
    ('2017_2018_2019_2020_2021',        'GRU-Counts'),
    ('2017_2018_2019_2020_2021',        'Transf-Counts'),
    ('2017_2018_2019_2020_2021',       'GRU-Transf'),
    ('2017_2018_2019_2020_2021',     'GRU-GRU_ETE'),
    ('2017_2018_2019_2020_2021', 'Transf-Transf_ETE')
]]
df_id.reset_index().to_csv('tables/id_table.csv')
df_id

Year_Group                                   2009_2010_2011_2012  \
Comparison                                            GRU-Counts   
Task               Metric                                          
hospital_mortality auc                       0.058 (0.041,0.077)   
                   auprc_c                   0.245 (0.174,0.314)   
                   ace_abs_logistic_logit   0.001 (-0.001,0.003)   
LOS_7              auc                       0.057 (0.049,0.066)   
                   auprc_c                    0.138 (0.118,0.16)   
                   ace_abs_logistic_logit    0.005 (-0.001,0.01)   
readmission_30     auc                       0.004 (-0.011,0.02)   
                   auprc_c                 -0.022 (-0.052,0.006)   
                   ace_abs_logistic_logit  -0.004 (-0.008,0.001)   
icu_admission      auc                        0.074 (0.05,0.096)   
                   auprc_c                   0.149 (0.077,0.217)   
                   ace_abs_logistic_logit  -0.002 (-0.005,0.001)   

Year_Group                                                        \
Comparison                                         Transf-Counts   
Task               Metric                                          
hospital_mortality auc                       0.049 (0.034,0.064)   
                   auprc_c                    0.157 (0.105,0.21)   
                   ace_abs_logistic_logit     0.0 (-0.002,0.002)   
LOS_7              auc                        0.038 (0.03,0.047)   
                   auprc_c                   0.086 (0.066,0.106)   
                   ace_abs_logistic_logit   0.001 (-0.004,0.005)   
readmission_30     auc                       0.004 (-0.012,0.02)   
                   auprc_c                 -0.014 (-0.042,0.016)   
                   ace_abs_logistic_logit   -0.004 (-0.007,-0.0)   
icu_admission      auc                       0.061 (0.038,0.082)   
                   auprc_c                  0.042 (-0.018,0.103)   
                   ace_abs_logistic_logit  -0.001 (-0.005,0.002)   

Year_Group                                                        \
Comparison                                            GRU-Transf   
Task               Metric                                          
hospital_mortality auc                       0.009 (0.001,0.018)   
                   auprc_c                   0.088 (0.024,0.148)   
                   ace_abs_logistic_logit   0.001 (-0.002,0.003)   
LOS_7              auc                       0.019 (0.013,0.025)   
                   auprc_c                   0.052 (0.035,0.068)   
                   ace_abs_logistic_logit   0.005 (-0.001,0.009)   
readmission_30     auc                       -0.0 (-0.014,0.013)   
                   auprc_c                 -0.008 (-0.035,0.017)   
                   ace_abs_logistic_logit    -0.0 (-0.002,0.002)   
icu_admission      auc                       0.013 (0.001,0.025)   
                   auprc_c                   0.106 (0.048,0.161)   
                   ace_abs_logistic_logit  -0.001 (-0.002,0.001)   

Year_Group                                                         \
Comparison                                            GRU-GRU_ETE   
Task               Metric                                           
hospital_mortality auc                       0.006 (-0.005,0.016)   
                   auprc_c                    0.017 (-0.046,0.08)   
                   ace_abs_logistic_logit  -0.004 (-0.006,-0.003)   
LOS_7              auc                      -0.002 (-0.007,0.004)   
                   auprc_c                   0.002 (-0.014,0.019)   
                   ace_abs_logistic_logit  -0.015 (-0.023,-0.005)   
readmission_30     auc                        0.017 (0.003,0.032)   
                   auprc_c                    0.033 (0.009,0.059)   
                   ace_abs_logistic_logit   -0.009 (-0.01,-0.005)   
icu_admission      auc                        0.041 (0.025,0.059)   
                   auprc_c              

In [8]:
df_ood = pd.read_csv("tables/ood.csv")

df_ood = (
    df_ood
    .groupby(['Year_Group','Comparison','Task','Metric'])['Delta']
    .quantile([0.025,0.5,0.975])
    .unstack()
    .reset_index()
    .query("Metric==['auc','auprc_c','ace_abs_logistic_logit']")
    .rename(columns={0.5:'mid',0.025:'lower',0.975:'upper'})
)

df_ood = (
    df_ood
    .assign(
        Delta = df_ood['mid'].round(3).astype(str) + 
        " (" + df_ood['lower'].round(3).astype(str) + "," + 
        df_ood['upper'].round(3).astype(str) + ")"
    )
    .pivot_table(
        columns=['Year_Group','Comparison'],
        index=['Task','Metric'],
        values='Delta',
        aggfunc='first'
    )
    .reindex(labels=tasks,level=0)
    .reindex(labels=['auc','auprc_c','ace_abs_logistic_logit'],level=1)
)

df_ood = df_ood[[
    (     '2013_2014_2015_2016',        'GRU-Counts'),
    (     '2013_2014_2015_2016',        'Transf-Counts'),
    (     '2013_2014_2015_2016',       'GRU-Transf'),
    (     '2013_2014_2015_2016',     'GRU-GRU_ETE'),
    (     '2013_2014_2015_2016', 'Transf-Transf_ETE'),
    ('2017_2018_2019_2020_2021',        'GRU-Counts'),
    ('2017_2018_2019_2020_2021',        'Transf-Counts'),
    ('2017_2018_2019_2020_2021',       'GRU-Transf'),
    ('2017_2018_2019_2020_2021',     'GRU-GRU_ETE'),
    ('2017_2018_2019_2020_2021', 'Transf-Transf_ETE')
]]

df_ood.reset_index().to_csv('tables/ood_table.csv')
df_ood

Year_Group                                    2013_2014_2015_2016  \
Comparison                                             GRU-Counts   
Task               Metric                                           
hospital_mortality auc                       0.012 (-0.013,0.039)   
                   auprc_c                   0.084 (-0.013,0.178)   
                   ace_abs_logistic_logit   -0.001 (-0.004,0.002)   
LOS_7              auc                         0.02 (0.007,0.034)   
                   auprc_c                   0.029 (-0.001,0.059)   
                   ace_abs_logistic_logit  -0.024 (-0.031,-0.015)   
readmission_30     auc                      -0.004 (-0.032,0.023)   
                   auprc_c                   0.027 (-0.012,0.066)   
                   ace_abs_logistic_logit       0.006 (0.0,0.012)   
icu_admission      auc                       0.004 (-0.029,0.034)   
                   auprc_c                   0.045 (-0.047,0.136)   
                   ace_abs_logistic_logit      0.0 (-0.004,0.004)   

Year_Group                                                        \
Comparison                                         Transf-Counts   
Task               Metric                                          
hospital_mortality auc                      0.005 (-0.015,0.027)   
                   auprc_c                   0.023 (-0.05,0.102)   
                   ace_abs_logistic_logit   0.001 (-0.001,0.003)   
LOS_7              auc                       0.018 (0.005,0.031)   
                   auprc_c                   0.011 (-0.018,0.04)   
                   ace_abs_logistic_logit   0.004 (-0.004,0.012)   
readmission_30     auc                      0.007 (-0.018,0.031)   
                   auprc_c                  0.033 (-0.004,0.073)   
                   ace_abs_logistic_logit    0.005 (0.001,0.009)   
icu_admission      auc                      0.015 (-0.014,0.045)   
                   auprc_c                   0.104 (0.019,0.192)   
                   ace_abs_logistic_logit  -0.003 (-0.007,0.001)   

Year_Group                                                         \
Comparison                                             GRU-Transf   
Task               Metric                                           
hospital_mortality auc                       0.007 (-0.004,0.018)   
                   auprc_c                   0.059 (-0.028,0.135)   
                   ace_abs_logistic_logit   -0.002 (-0.005,0.001)   
LOS_7              auc                        0.002 (-0.006,0.01)   
                   auprc_c                   0.018 (-0.007,0.042)   
                   ace_abs_logistic_logit  -0.027 (-0.034,-0.019)   
readmission_30     auc                      -0.011 (-0.035,0.015)   
                   auprc_c                  -0.007 (-0.045,0.032)   
                   ace_abs_logistic_logit    0.001 (-0.001,0.004)   
icu_admission      auc                      -0.011 (-0.027,0.005)   
                   auprc_c                  -0.062 (-0.145,0.023)   
                   ace_abs_logistic_logit     0.003 (0.001,0.005)   

Year_Group                                                        \
Comparison                                           GRU-GRU_ETE   
Task               Metric                                          
hospital_mortality auc                      0.005 (-0.008,0.018)   
                   auprc_c                 -0.011 (-0.096,0.067)   
                   ace_abs_logistic_logit   0.003 (-0.001,0.006)   
LOS_7              auc                        0.0 (-0.008,0.009)   
                   auprc_c                 -0.016 (-0.037,0.006)   
                   ace_abs_logistic_logit   -0.006 (-0.02,0.007)   
readmission_30     auc                     -0.011 (-0.036,0.013)   
                   auprc_c                    0.0 (-0.035,0.035)   
                   ace_abs_logistic_logit     0.01 (0.006,0.013)   
icu_admission      auc                     -0.021 (-0.043,0.002)   
                   aupr